In [ ]:
@file:DependsOn("/data/tools/actin-personalization/actin-personalization.jar")

import com.hartwig.actin.personalization.ncr.serialization.NcrDataReader

val records = NcrDataReader.read("/data/patient_like_me/ncr/K2400223.csv")

In [ ]:
import com.hartwig.actin.personalization.datamodel.DistantMetastasesStatus
import com.hartwig.actin.personalization.datamodel.Episode
// import com.hartwig.actin.personalization.datamodel.LocationGroup
import com.hartwig.actin.personalization.datamodel.ReferencePatient
import com.hartwig.actin.personalization.datamodel.Treatment
import com.hartwig.actin.personalization.ncr.interpretation.ReferencePatientFactory
// import com.hartwig.actin.personalization.datamodel.TreatmentGroup
// import com.hartwig.actin.personalization.datamodel.serialization.ReferencePatientJson
// import com.hartwig.actin.personalization.similarity.population.DiagnosisAndEpisode
// import com.hartwig.actin.personalization.similarity.population.PatientPopulationBreakdown
// import com.hartwig.actin.personalization.similarity.population.PersonalizedDataAnalysis
// import io.github.oshai.kotlinlogging.KotlinLogging
// import com.hartwig.actin.personalization.similarity.population.PopulationDefinition

fun Episode.doesNotIncludeAdjuvantOrNeoadjuvantTreatment(): Boolean {
    return !hasHadPreSurgerySystemicChemotherapy &&
            !hasHadPostSurgerySystemicChemotherapy &&
            !hasHadPreSurgerySystemicTargetedTherapy &&
            !hasHadPostSurgerySystemicTargetedTherapy
}

val patients = ReferencePatientFactory.default().create(records)

val referencePop = patients.flatMap(ReferencePatient::tumorEntries).map { (diagnosis, episodes) ->
    diagnosis to episodes.single { it.order == 1 }
}
    .filter { (_, episode) ->
        episode.distantMetastasesStatus == DistantMetastasesStatus.AT_START &&
        // episode.systemicTreatmentPlan?.treatment?.let { it != Treatment.OTHER } == true &&
        episode.surgeries.isEmpty() &&
        episode.doesNotIncludeAdjuvantOrNeoadjuvantTreatment() &&
        episode.systemicTreatmentPlan?.observedPfsDays != null
    }
    .sortedBy { it.second.systemicTreatmentPlan!!.observedPfsDays!! }

val patientsByTreatment = referencePop.groupBy { (_, episode) ->
    episode.systemicTreatmentPlan!!.treatment.treatmentGroup
}
    .toList()
    .sortedByDescending { it.second.size }

In [ ]:
import com.hartwig.actin.personalization.similarity.population.DiagnosisAndEpisode

data class EventCountAndSurvivalAtTime(val daysSincePlanStart: Int, val numEvents: Int, val survival: Double)
    
tailrec fun eventAndCensorshipHistories(
    populationToProcess: List<DiagnosisAndEpisode>,
    eventHistory: List<EventCountAndSurvivalAtTime> = emptyList(),
    censorshipHistory: List<EventCountAndSurvivalAtTime> = emptyList()
): Pair<List<EventCountAndSurvivalAtTime>, List<EventCountAndSurvivalAtTime>> {
    return if (populationToProcess.isEmpty()) {
        eventHistory to censorshipHistory
    } else {
        val treatmentDetails = populationToProcess.first().second.systemicTreatmentPlan!!
        val previousEvent = eventHistory.lastOrNull() ?: EventCountAndSurvivalAtTime(0, 0, 1.0)
        val (newEventHistory, newCensorshipHistory) = if (treatmentDetails.hadProgressionEvent!!) {
            val newEvent = EventCountAndSurvivalAtTime(
                treatmentDetails.observedPfsDays!!, previousEvent.numEvents + 1, previousEvent.survival * (1 - (1.0 / populationToProcess.size))
            )
            Pair(eventHistory + newEvent, censorshipHistory)
        } else {
            val newCensorship = EventCountAndSurvivalAtTime(treatmentDetails.observedPfsDays!!, previousEvent.numEvents, previousEvent.survival)
            Pair(eventHistory, censorshipHistory + newCensorship)
        }
        eventAndCensorshipHistories(populationToProcess.drop(1), newEventHistory, newCensorshipHistory)
    }
}

data class EventCountAndPopulationAtTime(val daysSincePlanStart: Int, val numEvents: Int, val uncensoredSize: Int)

tailrec fun incorrectEventAndCensorshipHistories(
    populationToProcess: List<DiagnosisAndEpisode>,
    eventHistory: List<EventCountAndPopulationAtTime> = emptyList(),
    censorshipHistory: List<EventCountAndPopulationAtTime> = emptyList(),
    uncensoredSize: Int = populationToProcess.size
): Pair<List<EventCountAndPopulationAtTime>, List<EventCountAndPopulationAtTime>> {
    return if (populationToProcess.isEmpty()) {
        eventHistory to censorshipHistory
    } else {
        val treatmentDetails = populationToProcess.first().second.systemicTreatmentPlan!!
        val previousEvent = eventHistory.lastOrNull() ?: EventCountAndPopulationAtTime(0, 0, uncensoredSize)
        val (newEventHistory, newCensorshipHistory, newUncensoredSize) = if (treatmentDetails.hadProgressionEvent!!) {
            Triple(
                eventHistory + EventCountAndPopulationAtTime(treatmentDetails.observedPfsDays!!, previousEvent.numEvents + 1, uncensoredSize),
                censorshipHistory,
                uncensoredSize
            )
        } else {
            Triple(
                eventHistory,
                censorshipHistory + EventCountAndPopulationAtTime(treatmentDetails.observedPfsDays!!, previousEvent.numEvents, previousEvent.uncensoredSize),
                uncensoredSize - 1
            )
        }
        incorrectEventAndCensorshipHistories(populationToProcess.drop(1), newEventHistory, newCensorshipHistory, newUncensoredSize)
    }
}

val (eventHistory, censorshipHistory) = eventAndCensorshipHistories(referencePop)

// val survivorAndPopulationVector = referencePop.runningFold(Triple(0, 0, referencePop.size)) { (_, prevEvents, prevPopulation), curr ->
//     val treatmentDetails = curr.second.systemicTreatmentPlan!!
//     val (newEvents, newPopulation) = if (treatmentDetails.hadProgressionEvent!!) {
//         prevEvents + 1 to prevPopulation
//     } else {
//         prevSurvivors to prevPopulation - 1
//     Triple(treatmentDetails.observedPfsDays!!, treatmentDetails.hadProgressionEvent!! 

In [ ]:
%use kandy

In [ ]:
eventHistory.take(10)

In [ ]:
censorshipHistory.take(10)

In [ ]:
referencePop.size

In [ ]:
plot {
    step {
        x(eventHistory.map(EventCountAndSurvivalAtTime::daysSincePlanStart), "days")
        y(eventHistory.map(EventCountAndSurvivalAtTime::survival), "survival")
        color(eventHistory.map { "A" })
    }
}

In [ ]:
eventHistory.asSequence().filter { it.survival >= 0.499 && it.survival < 0.501 }.take(100).toList()